In [ ]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import os

In [ ]:
# Load the trained model
model_path = os.path.join("models", "best.pt")
model = YOLO(model_path)

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import threading
from ultralytics import YOLO
# ===== Detection summary box =====
from tkinter import scrolledtext
from collections import Counter
import torch



# Load YOLOv8 model
model = YOLO("models/best.pt")  # Update this path as needed

def update_detection_summary(results):
    try:
        # Get predicted class indices from YOLO result
        cls_tensor = results[0].boxes.cls
        if isinstance(cls_tensor, torch.Tensor):
            cls_list = cls_tensor.tolist()
        else:
            cls_list = cls_tensor

        # Count detections by class index
        counts = Counter(cls_list)

        # Map class index to name
        summary = ""
        for class_id, count in counts.items():
            class_name = model.names[int(class_id)]
            summary += f"{class_name}: {count}\n"

        # Update text box
        detection_box.config(state=tk.NORMAL)
        detection_box.delete(1.0, tk.END)
        detection_box.insert(tk.END, summary.strip() if summary else "No detections")
        detection_box.config(state=tk.DISABLED)
    except Exception as e:
        detection_box.config(state=tk.NORMAL)
        detection_box.delete(1.0, tk.END)
        detection_box.insert(tk.END, f"Error: {str(e)}")
        detection_box.config(state=tk.DISABLED)

# GUI setup
root = tk.Tk()
root.title("YOLOv8 GUI Demo")
root.geometry("1024x768")

# State variables
cap = None
running = False

# Helper: Show image in GUI
def show_image(img_cv2):
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_tk = ImageTk.PhotoImage(img_pil)
    panel.imgtk = img_tk
    panel.configure(image=img_tk)

# Prediction: Run YOLO on static image
def predict_image(path):
    results = model(path)
    img = results[0].plot()
    show_image(img)
    update_detection_summary(results)

# Prediction: Open image
def upload_image():
    path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if path:
        predict_image(path)

# Webcam handling
def webcam_loop():
    global cap, running
    while running:
        ret, frame = cap.read()
        if not ret:
            break

        # 👇 Copy and flip the frame horizontally (mirror)
        frame_flipped = cv2.flip(frame.copy(), 1)  # 1 = horizontal flip

        # Run YOLOv8 on the flipped frame
        results = model(frame_flipped)

        # Draw bounding boxes
        img = results[0].plot()

        # Display
        show_image(img)
        update_detection_summary(results)

        if cv2.waitKey(1) == ord('q'):
            break



# Start webcam
def start_webcam():
    global cap, running
    stop_all_streams()  # Stop other sources
    cap = cv2.VideoCapture(0)
    running = True
    threading.Thread(target=webcam_loop, daemon=True).start()

# Video playback loop
def video_loop(path):
    global cap, running
    cap = cv2.VideoCapture(path)
    running = True
    while running and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        img = results[0].plot()
        show_image(img)
        update_detection_summary(results)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
    cap.release()

# Upload and play video
def upload_video():
    path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
    if path:
        stop_all_streams()
        threading.Thread(target=video_loop, args=(path,), daemon=True).start()

# Stop all video/camera streams
def stop_all_streams():
    global cap, running
    running = False
    if cap:
        cap.release()
        cap = None

# Buttons
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

# ===== Button Panel on the Left =====
button_panel = tk.Frame(root, width=200, bg='lightgray')
button_panel.pack(side="left", fill="y")

btn_upload_img = tk.Button(button_panel, text="Upload Image", command=upload_image)
btn_upload_img.pack(pady=10, padx=10, fill="x")

btn_upload_vid = tk.Button(button_panel, text="Upload Video", command=upload_video)
btn_upload_vid.pack(pady=10, padx=10, fill="x")

btn_cam_start = tk.Button(button_panel, text="Start Webcam", command=start_webcam)
btn_cam_start.pack(pady=10, padx=10, fill="x")

btn_cam_stop = tk.Button(button_panel, text="Stop Webcam", command=stop_all_streams)
btn_cam_stop.pack(pady=10, padx=10, fill="x")


detection_box = scrolledtext.ScrolledText(button_panel, height=15, width=25, wrap=tk.WORD)
detection_box.pack(pady=10, padx=10, fill="both")
detection_box.insert(tk.END, "Detection summary will appear here.")
detection_box.config(state=tk.DISABLED)

# ===== Display Panel on the Right =====
display_panel = tk.Frame(root, bg='black')
display_panel.pack(side="right", expand=True, fill="both")

panel = tk.Label(display_panel, bg='black')
panel.pack(expand=True)

# Run GUI
root.mainloop()
